In [1]:
from spacy import load
model = load('en_core_web_sm')
print('loaded spacy model')

loaded spacy model


In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/inshorts-dataset-english/english_news_dataset.csv', parse_dates=['Date'], dayfirst=True)
df['headline'] = df['Headline'].str.lower()
print(df.shape)

(43610, 4)


In [12]:
from arrow import now
time_start = now()
nouns = []
for index, headline in enumerate(df['headline'].values.tolist()):
    tokens = model(headline)
    for token in tokens:
        if token.pos_ in {'NOUN', 'PROPN'}:
            nouns.append(token.text)
    if index > 0 and index % 10000 == 0:
        print(len(nouns))
nouns = set(nouns)
print(now() - time_start)

55037
109900
164929
220887
0:04:55.858422


In [13]:
len(nouns)

7680

In [3]:
parts = {}
for index, headline in enumerate(df['Headline'].values.tolist()):
    tokens = model(headline)
    for token in tokens:
        if token.text in parts:
            parts[token].append(token.pos_)
        else:
            parts[token] = [token.pos_]
    if index > 0 and index % 10000 == 0:
        print(len(parts))
print(len(parts))

130953
262443
394304
527092
574932


In [5]:
def get_parts(arg: str):
    tokens = model(arg)
    return [(token.text, token.pos_) for token in tokens]
df['Headline_parts'] = df['Headline'].apply(get_parts)
df.head()['Headline_parts']

0    [(Congress, PROPN), (leader, NOUN), (Baljinder...
1    [(17, NUM), (-, PUNCT), (year, NOUN), (-, PUNC...
2    [(Hampers, NOUN), (to, PART), (welcome, VERB),...
3    [(Only, ADV), (10, NUM), (%, NOUN), (women, NO...
4    [(Ganesh, ADJ), (temple, NOUN), (decorated, VE...
Name: Headline_parts, dtype: object

In [6]:
from arrow import now
time_start  = now()
df['headline'] = df['Headline'].str.lower()
df['headline_parts'] = df['headline'].apply(get_parts)
print(now() - time_start)

0:05:07.253149


In [7]:
df.head(n=10)['headline_parts']

0    [(congress, PROPN), (leader, NOUN), (baljinder...
1    [(17, NUM), (-, PUNCT), (year, NOUN), (-, PUNC...
2    [(hampers, VERB), (to, PART), (welcome, VERB),...
3    [(only, ADV), (10, NUM), (%, NOUN), (women, NO...
4    [(ganesh, ADJ), (temple, PROPN), (decorated, V...
5    [(bee, PROPN), (attack, NOUN), (during, ADP), ...
6    [(manipur, PROPN), (govt, PROPN), (forms, PROP...
7    [(cabinet, NOUN), (approves, VERB), (bill, NOU...
8    [(wo, AUX), (n't, PART), (accept, VERB), (his,...
9    [(heavy, ADJ), (to, ADP), (very, ADV), (heavy,...
Name: headline_parts, dtype: object

In [9]:
parts_list = [(key, value) for key, value in parts.items()]
parts_list[:10]

[(Congress, ['PROPN']),
 (leader, ['NOUN']),
 (Baljinder, ['PROPN']),
 (Singh, ['PROPN']),
 (shot, ['VERB']),
 (dead, ['ADJ']),
 (at, ['ADP']),
 (home, ['NOUN']),
 (in, ['ADP']),
 (Punjab, ['PROPN'])]

In [11]:
{key: len(value) for key, value in parts.items() if len(value) > 1}

{}

In [ ]:
from collections import Counter

def get_noun_percentage(counter: dict) -> float:
    result = 0
    if 'NOUN' in counter.keys():
        result += counter['NOUN']
    if 'PROPN' in counter.keys():
        result += counter['PROPN']
    return result / sum(list(counter.values()))        
    
counters = {key: dict(Counter(value)) for key, value in parts.items()}
noun_percentages = {key: get_noun_percentage(value) for key, value in counters.items()}
noun_percent_df = pd.DataFrame(data={'token': list(noun_percentages.keys()), 'percent': list(noun_percentages.values())})
noun_percent_df.sort_values(ascending=False, by='percent')

In [ ]:
noun_percent_df['percent'].value_counts()

In [ ]:
df[['Headline']].head()

In [ ]:
for index, row in df.head().iterrows():
    tokens = model(row['Headline'])
    for token in tokens:
        print(token.text, token.pos_)